In [7]:
!pip install metpy


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import glob
import pandas as pd
from zipfile import ZipFile
from datetime import datetime
from metpy.io.metar import *

In [2]:
if 'data-science-challenge-at-eef-2024.zip' not in glob.glob('*'):
    !kaggle competitions download -c data-science-challenge-at-eef-2024

    with ZipFile('data-science-challenge-at-eef-2024.zip') as zip:
        zip.extractall()

In [3]:
df = pd.read_csv('public.csv')
df.head()

,flightid,hora_ref,origem,destino,url_img_satelite,metaf,metar,prev_troca_cabeceira,troca_cabeceira_hora_anterior,espera
0,504a62621cd231d6ab67e674ce538cd3,2022-06-01T01:00:00Z,SBCF,SBFL,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBFL 010000Z 17009KT 140V200 9999 BKN030...,0,1,0.0
1,b0fd0f83644625ecc21f5261e8e5e347,2022-06-01T01:00:00Z,SBPA,SBFL,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBFL 010000Z 17009KT 140V200 9999 BKN030...,0,1,0.0
2,1210f0ca07ddca00d09a3e02d3b100d8,2022-06-01T01:00:00Z,SBSP,SBCF,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBCF 010000Z 12006KT CAVOK 21/14 Q1018=,0,0,0.0
3,b25032f34507cce285ee779446496568,2022-06-01T01:00:00Z,SBGR,SBCT,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBCT 010000Z 10006KT 7000 -RA BKN004 OVC...,0,0,0.0
4,00762a9892ecba7c66d1d87800d38cac,2022-06-01T01:00:00Z,SBSP,SBSV,http://satelite.cptec.inpe.br/repositoriogoes/...,NaN,METAR SBSV 010000Z 11008KT 9999 FEW023 27/21 Q...,0,1,0.0


In [9]:
def convert_metar(col):
    # print(col)
    metar, date = col
    if None not in [metar, date]:
        date = datetime.strptime(str(date)[:10], '%Y-%m-%d')
        return parse_metar_to_dataframe(metar, year=date.year, month=date.month).values[0]
    
def metar_cols(col):
    metar, date = col
    if None not in [metar, date]:
        date = datetime.strptime(str(date)[:10], '%Y-%m-%d')
        return parse_metar_to_dataframe(metar, year=date.year, month=date.month).columns

In [125]:
metar = df[['metar', 'hora_ref']].loc[~df['metar'].isna()]
metar_data = metar.apply(convert_metar, axis=1).tolist()
df_metar = pd.DataFrame(metar_data)
df_metar.columns = metar_cols(metar.iloc[0])
df_metar

,station_id,latitude,longitude,elevation,date_time,wind_direction,wind_speed,wind_gust,visibility,current_wx1,current_wx2,current_wx3,low_cloud_type,low_cloud_level,medium_cloud_type,medium_cloud_level,high_cloud_type,high_cloud_level,highest_cloud_type,highest_cloud_level,cloud_coverage,air_temperature,dew_point_temperature,altimeter,current_wx1_symbol,current_wx2_symbol,current_wx3_symbol,remarks,air_pressure_at_sea_level,eastward_wind,northward_wind
0,SBFL,-27.67,-48.53,5,2022-06-01,170.0,9.0,NaN,9999,NaN,NaN,NaN,BKN,3000.0,NaN,NaN,NaN,NaN,NaN,NaN,6,14.0,7.0,30.12,0,0,0,,1020.30,-1.562834,8.863270
1,SBFL,-27.67,-48.53,5,2022-06-01,170.0,9.0,NaN,9999,NaN,NaN,NaN,BKN,3000.0,NaN,NaN,NaN,NaN,NaN,NaN,6,14.0,7.0,30.12,0,0,0,,1020.30,-1.562834,8.863270
2,SBCF,-19.63,-43.97,828,2022-06-01,120.0,6.0,NaN,10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,21.0,14.0,30.06,0,0,0,,1015.39,-5.196152,3.000000
3,SBCT,-25.52,-49.17,908,2022-06-01,100.0,6.0,NaN,7000,-RA,NaN,NaN,BKN,400.0,OVC,1200.0,NaN,NaN,NaN,NaN,8,10.0,10.0,30.09,61,0,0,,1020.20,-5.908847,1.041889
4,SBSV,-12.90,-38.32,6,2022-06-01,110.0,8.0,NaN,9999,NaN,NaN,NaN,FEW,2300.0,NaN,NaN,NaN,NaN,NaN,NaN,2,27.0,21.0,29.94,0,0,0,,1014.27,-7.517541,2.736161
5,SBPA,-30.00,-51.17,3,2022-06-01,NaN,3.0,NaN,10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,9.0,4.0,30.15,0,0,0,,1021.31,NaN,NaN
6,SBBR,-15.87,-47.92,1061,2022-06-01,70.0,2.0,NaN,10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,21.0,8.0,30.06,0,0,0,,1014.23,-1.879385,-0.684040
7,SBGR,-23.42,-46.47,750,2022-06-01,130.0,5.0,NaN,9999,NaN,NaN,NaN,OVC,900.0,NaN,NaN,NaN,NaN,NaN,NaN,8,16.0,15.0,30.09,0,0,0,,1018.29,-3.830222,3.213938
8,SBPA,-30.00,-51.17,3,2022-06-01,NaN,3.0,NaN,10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,9.0,4.0,30.15,0,0,0,,1021.31,NaN,NaN
9,SBGR,-23.42,-46.47,750,2022-06-01,130.0,5.0,NaN,9999,NaN,NaN,NaN,OVC,900.0,NaN,NaN,NaN,NaN,NaN,NaN,8,16.0,15.0,30.09,0,0,0,,1018.29,-3.830222,3.213938


In [ ]:
metaf = df[['metaf', 'hora_ref']].loc[~df['metaf'].isna()]
metaf_data = metaf.apply(convert_metar, axis=1).tolist()
df_metaf = pd.DataFrame(metaf_data)
df_metaf.columns = metar_cols(metaf.iloc[0])
df_metar